This function computes the singular value decomposition (SVD) of $ 2\times 2$ deformation gradient $ \mathbf{\nabla F_{t_0}^t(\mathbf{x}_0)} $:

\begin{equation}
\mathbf{\nabla F}_{t_0}^t(\mathbf{x}_0) = \begin{pmatrix} a && b \\ c && d \end{pmatrix} = \mathbf{P}_{t_0}^t \Sigma_{t_0}^t {\mathbf{Q}_{t_0}^t}^T
\end{equation}

For simplicity we now drop the time-interval when expressing $ \mathbf{\nabla F}, \mathbf{P}, \mathbf{\Sigma} $ and $ \mathbf{Q} $

The decomposition is given by:

\begin{equation}
\mathbf{\nabla F}_{t_0}^t = \begin{pmatrix} \cos(\phi) && -\sin(\phi) \\ \sin(\phi) && \cos(\phi) \end{pmatrix} \begin{pmatrix} \sigma_1 && 0 \\ 0 && \sigma_2 \end{pmatrix}\begin{pmatrix} \cos(\theta) && -\sin(\theta) \\ \sin(\theta) && \cos(\theta) \end{pmatrix}.
\end{equation}

Computation of $ P $:

\begin{equation}
\phi = \dfrac{1}{2} \tan^{-1}(\dfrac{2ac+2bd}{a^2+b^2-c^2-d^2})
\end{equation}
and 
\begin{equation}
\mathbf{P} = \begin{pmatrix} \cos(\phi) && -\sin(\phi) \\ \sin(\phi) && \cos(\phi) \end{pmatrix}
\end{equation}

Computation of $ Q $:

\begin{equation}
\theta = \dfrac{1}{2} \tan^{-1}(\dfrac{2ab+2cd}{a^2-b^2+c^2-d^2})
\end{equation}
and 
\begin{equation}
\mathbf{Q} = \begin{pmatrix} sign(s_{1})cos(\theta) && -sign(s_{2})\sin(\theta) \\ sign(s_{1})\sin(\theta) && sign(s_{2})\cos(\theta) \end{pmatrix},
\end{equation} with

\begin{align*}
s_{1} &= (a\cos(\phi)+c\sin(\phi))\cos(\theta) + (b\cos(\phi)+d\sin(\phi))\sin(\theta) \\
s_{2} &= (a\sin(\phi)-c\sin(\phi))\cos(\theta) + (-b\sin(\phi)+d\cos(\phi))\sin(\theta)
\end{align*}

Computation of $ S $:
\begin{align*}
\sigma_1 &= \sqrt{\dfrac{S_1+S_2}{2}} \\
\sigma_2 &= \sqrt{\dfrac{S_1-S_2}{2}},
\end{align*} with 

\begin{align*}
S_1 &= a^2+b^2+c^2+d^2 \\
S_2 &= \sqrt{(a^2+b^2-c^2-d^2)^2+4(ac+bd)^2}
\end{align*}

\begin{equation}
\Sigma = \begin{pmatrix} \sigma_1 && 0 \\ 0 && \sigma_2 \end{pmatrix}
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| gradFmap | array (2, 2) | $ \mathbf{\nabla F}_{t_0}^t(\mathbf{x}_0) $|
| P | array (2, 2) | $ \mathbf{P}_{t_0}^t(\mathbf{x}_0) $: orthogonal rotation matrix |
| Q | array (2, 2) | $ \mathbf{Q}_{t_0}^t(\mathbf{x}_0) $: orthogonal rotation matrix |
| SIG | array (2, 2) | $ \mathbf{\Sigma}_{t_0}^t(\mathbf{x}_0) $: positive definite diagonal matrix representing uniaxial stretching/compression  |

In [1]:
# Import numpy
import numpy as np

# Import math tools
from math import sqrt, cos, sin, atan2

In [2]:
def SVD(gradFmap):
    '''Compute the singular value decomposition. For an arbitrary matrix F, decomposite as F = P \Sigma Q^T. 
    
    Parameters:
        gradFmap: array (2,2) arbitrary 2 by 2 matrix
    
    Returns:
        P: orthogonal rotation tensor
        Sig: array(2,2) diagonal matrix of singular values
        Q: orthogonal rotation tensor
    '''
    Su = gradFmap@gradFmap.transpose()
    theta = 0.5*atan2(Su[0,1]+Su[1,0], Su[0,0]-Su[1,1])
    Ctheta = cos(theta)
    Stheta = sin(theta)
    P = np.array([[Ctheta, -Stheta], [Stheta, Ctheta]])

    Sw = gradFmap.transpose()@gradFmap
    phi = 0.5*atan2((Sw[0,1]+Sw[1,0]), (Sw[0,0]-Sw[1,1]))
    Cphi = cos(phi)
    Sphi = sin(phi)
    W = np.array([[Cphi, -Sphi], [Sphi, Cphi]])

    SUsum= Su[0,0]+Su[1,1]
    SUdif= sqrt((Su[0,0]-Su[1,1])**2 + 4*(Su[0,1]*Su[1,0]))
    
    if SUsum-SUdif < 0: # This happens due to numerical inaccuracies
        svals = np.array([sqrt((SUsum+SUdif)/2), 0])
    
    else:
        svals= np.array([sqrt((SUsum+SUdif)/2), sqrt((SUsum-SUdif)/2)])
   
    SIG = np.diag(svals)

    S = P.transpose()@gradFmap@W
    C = np.diag([np.sign(S[0,0]), np.sign(S[1,1])])
    Q = W@C
    
    return P, SIG, Q